In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - calculate_derived_quantities.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Calculate quantities derived from the best-fits to the data using each 
model. 
- The alt/az of the principal axis of the density ellipsoid.
- The triaxiality parameter for the density ellipsoid.
- fdisk in terms of fractional density to fdisk in terms of fractional number 
    of stars.
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, pdb, copy
import numpy as np

# Matplotlib and plotting 
import matplotlib
import matplotlib.pyplot as plt

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import util as putil
from ges_mass import plot as pplot

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

### Keywords, Pathing, Loading, Data Preparation

In [ ]:
# %load ../../src/nb_modules/keywords_pathing_loading_data_prep.py
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'FEH_MIN_GSE','FEH_MAX_GSE','DF_VERSION','KSF_VERSION','NPROCS',
            'RO','VO','ZO']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,feh_min_gse,feh_max_gse,df_version,\
    ksf_version,nprocs,ro,vo,zo = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]
feh_range_gse = [feh_min_gse,feh_max_gse]
feh_range_all = [feh_min,feh_max_gse]
# feh_range_fit = copy.deepcopy( # Need to choose here


## Pathing
fit_paths = putil.prepare_paths(base_dir,apogee_dr,apogee_results_vers,gaia_dr,
                                df_version,ksf_version)
data_dir,version_dir,ga_dir,gap_dir,df_dir,ksf_dir,fit_dir = fit_paths

## Filenames
fit_filenames = putil.prepare_filenames(ga_dir,gap_dir,feh_range_gse)
apogee_SF_filename,apogee_effSF_filename,apogee_effSF_mask_filename,\
    iso_grid_filename,clean_kinematics_filename = fit_filenames

## File loading and data preparation
fit_stuff,other_stuff = putil.prepare_fitting(fit_filenames,
    [ndmod,dmod_min,dmod_max], ro,zo,return_other=True)
apogee_effSF_mask,dmap,iso_grid,jkmins,dmods,ds,effsel_grid,apof,\
    allstar_nomask,orbs_nomask = fit_stuff
Rgrid,phigrid,zgrid = effsel_grid

# ## Load the distribution functions
# df_filename = df_dir+'dfs.pkl'
# betas = [0.3,0.8]
# dfs = putil.load_distribution_functions(df_filename, betas)

In [ ]:
fig_dir = './fig/'

### Global Parameters

In [ ]:
# %load ../../src/nb_modules/global_fitting_params.py
## general kwargs
verbose = True

## HaloFit kwargs (ordering follows HaloFit.__init__)
# allstar and orbs loaded in prep cell
init = None
init_type = 'ML'
# fit_type provided at runtime
mask_disk = True
mask_halo = True
# densfunc, selec provided at runtime
# effsel, effsel_grid, effsel_mask, dmods loaded in prep cell
nwalkers = 100
nit = int(2e3)
ncut = int(1e3)
# usr_log_prior provided at runtime
n_mass = 5000 # int(nwalkers*(nit-ncut))
int_r_range = [2.,70.]
iso = None # Will read from iso_grid_filename
# iso_filename, jkmins loaded in prep cell
# feh_range provided at runtime
# logg_range loaded in config cell
# fit_dir, gap_dir, ksf_dir loaded in prep cell
# version provided at runtime
# ro, vo, zo loaded in config cell

hf_kwargs = {## HaloFit parameters
             'allstar':allstar_nomask,
             'orbs':orbs_nomask,
             'init':init,
             'init_type':init_type,
             # 'fit_type':fit_type, # provided at runtime
             'mask_disk':mask_disk,
             'mask_halo':mask_halo,
             ## _HaloFit parameters
             # 'densfunc':densfunc, # provided at runtime
             # 'selec':selec, # provided at runtime
             'effsel':apof,
             'effsel_mask':apogee_effSF_mask,
             'effsel_grid':effsel_grid,
             'dmods':dmods,
             'nwalkers':nwalkers,
             'nit':nit,
             'ncut':ncut,
             # 'usr_log_prior':usr_log_prior, # provided at runtime
             'n_mass':n_mass,
             'int_r_range':int_r_range,
             'iso':iso,
             'iso_filename':iso_grid_filename,
             'jkmins':jkmins,
             # 'feh_range':feh_range, # provided at runtime
             'logg_range':logg_range,
             'fit_dir':fit_dir,
             'gap_dir':gap_dir,
             'ksf_dir':ksf_dir,
             # 'version':version, # provided at runtime
             'verbose':verbose,
             'ro':ro,
             'vo':vo,
             'zo':zo}

## pmass.fit() function kwargs
# nprocs set in config file
force_fit = True
mle_init = True
just_mle = False
return_walkers = True
optimizer_method = 'Powell'
mass_int_type = 'spherical_grid'
batch_masses = True
make_ml_aic_bic = True
calculate_masses = True
post_optimization = True
mcmc_diagnostic = True

fit_kwargs = {# 'nprocs':nprocs, # Normally given at runtime 
              'force_fit':force_fit,
              'mle_init':mle_init,
              'just_mle':just_mle,
              'return_walkers':return_walkers,
              'optimizer_method':optimizer_method,
              'mass_int_type':mass_int_type,
              'batch_masses':batch_masses,
              'make_ml_aic_bic':make_ml_aic_bic,
              'calculate_masses':calculate_masses,
              'post_optimization':post_optimization,
              'mcmc_diagnostic':mcmc_diagnostic,
              }

### Convenience functions

In [ ]:
def calculate_principal_axis_altaz(hf,params=None,degrees=True,n=100):
    '''calculate_principal_axis_altaz:
    
    For a given model calculate the alt/az of the major axis.
    
    Args:
        hf (HaloFit): The HaloFit object.
        params (array): The parameters of the model. If None then use all
            the parameters from the HaloFit object.
        degrees (bool): If True then return alt/az in degrees.
        n (int): The number of sets of params to use to calculate the alt/az.
    
    Returns:
        alt (float): The altitude of the major axis.
        az (float): The azimuth of the major axis.
        '''
    if not hf._hasResults:
        hf.get_results()
    if params is None:
        params = hf.samples
    params = np.atleast_2d(params)
    nparams = params.shape[0]
    indx = np.random.choice(nparams,n,replace=False)
    alt = np.zeros(n)
    az = np.zeros(n)

    for i in range(n):
        pavec = hf.get_rotated_coords_in_gc_frame(params=params[indx[i]],
            vec=np.array([1,0,0]))[0]
        # pavec = pavec/np.linalg.norm(pavec)
        alt[i],az[i] = putil.vec_to_alt_az(pavec,degrees=degrees)
    
    return alt,az

def calculate_triaxiality_parameter(hf,params=None,n=None):
    '''calculate_triaxiality_parameter:

    Calculate the triaxiality parameter 

    Args:
        b (float) - ratio of Y to X axes (p in the paper)
        c (float) - ratio of Z to X axes (q in the paper

    Returns:
        T = triaxiality parameter
    '''
    if not hf._hasResults:
        hf.get_results()
    if params is None:
        params = hf.samples
    else:
        params = np.atleast_2d(params)
    if n is None:
        n = params.shape[0]
    p_indx,q_indx = pdens.get_densfunc_params_indx(hf.densfunc,['p','q'])
    nparams = params.shape[0]
    indx = np.random.choice(nparams,n,replace=False)
    ps = params[indx,p_indx]
    qs = params[indx,q_indx]
    abc_vec = np.array([np.ones_like(ps),ps,qs])
    c,b,a = np.sort(abc_vec,axis=0)
    return (1-(b/a)**2)/(1-(c/a)**2)

### Calculate masses within 55 kpc

In [ ]:
n_mass = int(1e4)
r_min = 2.
r_max = [55.,]

selecs = ['eLz','AD','JRLz',None]
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
            #]
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]
fit_types = ['gse','gse','gse','all']
versions = 4*['100w_1e4n']
hf_kwargs['verbose'] = True

for i,_selec in enumerate(selecs):
    print('\n',_selec,'\n----')
    for j,_densfunc in enumerate(densfuncs):
        print('\n',_densfunc.__name__,'\n----')
        if fit_types[i] == 'gse':
            _feh_range = copy.deepcopy(feh_range_gse)
        elif fit_types[i] == 'all':
            _feh_range = copy.deepcopy(feh_range_all)
        hf = pmass.HaloFit(densfunc=_densfunc,
                           selec=_selec,
                           fit_type=fit_types[i],
                           feh_range=_feh_range,
                           version=versions[i],
                           **hf_kwargs)
        hf.get_results()
        out = pmass.mass(hf,n_mass=n_mass,int_r_range=[r_min,r_max[0]],
                         nprocs=10)
        masses = out[0]
        if 'plusexpdisk' in _densfunc.__name__:
            masses = masses[0]
        mlow,mmed,mhigh = np.percentile(masses,[16,50,84])
        logmlow,logmmed,logmhigh = np.log10([mlow,mmed,mhigh])
        print('mass = {:.2f} +{:.2f} -{:.2f}'.format(mmed,mhigh-mmed,mmed-mlow))
        print('log10(mass) = {:.2f} +{:.2f} -{:.2f}'.format(logmmed,logmhigh-logmmed,logmmed-logmlow))

### Calculate the mass fraction within the break radius for the best AD and Halo models

In [ ]:
n_mass = int(1e4)
r_min = 2.

lb_densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
hb_densfunc = pdens.triaxial_broken_angle_zvecpa
r_cutoff_ind = pdens.get_densfunc_params_indx(hb_densfunc,['r1'])[0]
version = '100w_1e4n'

lb_hf = pmass.HaloFit(densfunc=lb_densfunc, 
                      selec=None,
                      fit_type='all',
                      feh_range=feh_range_all,
                      version='100w_1e4n',
                      **hf_kwargs)
lb_hf.get_results()
hb_hf = pmass.HaloFit(densfunc=hb_densfunc,
                      selec='AD',
                      fit_type='gse',
                      feh_range=feh_range_gse,
                      version='100w_1e4n',
                      **hf_kwargs)
hb_hf.get_results()

samples_for_mass = np.random.choice(lb_hf.samples.shape[0],n_mass,replace=False)

r_cutoffs = hb_hf.samples[samples_for_mass,r_cutoff_ind]
lb_mass = np.zeros(n_mass)
hb_mass = np.zeros(n_mass)

for i in range(n_mass):
    lb_hf_samples = np.atleast_2d(lb_hf.samples[samples_for_mass[i]])
    hb_hf_samples = np.atleast_2d(hb_hf.samples[samples_for_mass[i]])

    assert hb_hf_samples[0,r_cutoff_ind] == r_cutoffs[i]

    lb_out = pmass.mass(lb_hf,lb_hf_samples,int_r_range=[2.,r_cutoffs[i]],
                            n_mass=1, nprocs=1, verbose=False)
    hb_out = pmass.mass(hb_hf,hb_hf_samples,int_r_range=[2.,r_cutoffs[i]],
                            n_mass=1, nprocs=1, verbose=False)
    
    if 'plusexpdisk' in lb_densfunc.__name__:
        lb_mass[i] = lb_out[0][0][0]
    else:
        lb_mass[i] = lb_out[0][0]
    if 'plusexpdisk' in hb_densfunc.__name__:
        hb_mass[i] = hb_out[0][0][0]
    else:
        hb_mass[i] = hb_out[0][0]

    print('Calculated mass for sample {}/{}'.format(i+1,n_mass),end='\r')
    
mfm,mfl,mfh = np.percentile(hb_mass/lb_mass,[50,16,84])
print('\nMass fraction for hb/lb: {:.2f} +{:.2f} -{:.2f}'.format(mfm,mfh-mfm,mfm-mfl))


### Calculate alt-az for all GS/E models

In [ ]:
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
            #]
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]
selecs = ['eLz','AD','JRLz']
version = '100w_1e4n'
fit_type = 'gse'
feh_range_fit = copy.deepcopy(feh_range_gse)

for i in range(len(selecs)):
    for j in range(len(densfuncs)):
        hf = pmass.HaloFit(densfunc=densfuncs[j], fit_type=fit_type,
            version=version, selec=selecs[i], feh_range=feh_range_fit, 
            **hf_kwargs)
        alt,az = calculate_principal_axis_altaz(hf,degrees=True,n=10000)

        alt_low,alt_med,alt_high = np.percentile(alt,[16,50,84])
        az_low,az_med,az_high = np.percentile(az,[16,50,84])
        alt_low_err = alt_med-alt_low
        alt_high_err = alt_high-alt_med
        az_low_err = az_med-az_low
        az_high_err = az_high-az_med

        phi_indx = pdens.get_densfunc_params_indx(hf.densfunc,['phi'])
        phi = pdens.denormalize_parameters(hf.samples,hf.densfunc,
            phi_in_degr=True)[:,phi_indx]
        phi_low,phi_med,phi_high = np.percentile(phi,[16,50,84])

        print('\n')
        print('densfunc: {}'.format(hf.densfunc.__name__))
        print('selec: {}'.format(hf.selec))
        print('alt: {} +{} -{}'.format(alt_med,alt_high_err,alt_low_err))
        print('az: {} +{} -{}'.format(az_med,az_high_err,az_low_err))
        print('phi: {} +{} -{}'.format(phi_med,phi_high-phi_med,phi_med-phi_low))
        print('\n\n')

### Calculate the triaxiality parameter

In [ ]:
hf = pmass.HaloFit(densfunc=densfuncs[2], fit_type=fit_type,
            version=version, selec=selecs[1], feh_range=feh_range_fit, 
            **hf_kwargs)

In [ ]:
# # Some scratch work on triaxiality stuff
# p_indx,q_indx = pdens.get_densfunc_params_indx(hf.densfunc,['p','q'])
# hf.get_results()
# params = hf.samples
# ps,qs = params[:,p_indx],params[:,q_indx]
# abc_vec = np.array([np.ones_like(ps),ps,qs])
# abc_vec.shape
# print(ps)
# print(qs)
# np.median(ps)
# calculate_triaxiality_parameter(hf,[3.,10.,0.54,0.46,1.,1.,1.]) # SHould be ~ 0.89
# abc_vec
# np.sort(abc_vec,axis=0)

In [ ]:
densfuncs = [pdens.triaxial_single_angle_zvecpa,
             pdens.triaxial_single_angle_zvecpa_plusexpdisk,
             pdens.triaxial_single_cutoff_zvecpa,
             pdens.triaxial_single_cutoff_zvecpa_plusexpdisk,
             pdens.triaxial_broken_angle_zvecpa,
             pdens.triaxial_broken_angle_zvecpa_plusexpdisk,
            #]
             pdens.triaxial_double_broken_angle_zvecpa,
             pdens.triaxial_double_broken_angle_zvecpa_plusexpdisk]
selecs = ['eLz','AD','JRLz']
version = '100w_1e4n'
fit_type = 'gse'
feh_range_fit = copy.deepcopy(feh_range_gse)

for i in range(len(selecs)):
    print('\nselec: {}'.format(selecs[i]))
    print('-----------------')
    for j in range(len(densfuncs)):
        hf = pmass.HaloFit(densfunc=densfuncs[j], fit_type=fit_type,
            version=version, selec=selecs[i], feh_range=feh_range_fit, 
            **hf_kwargs)
        
        Ts = calculate_triaxiality_parameter(hf,n=100)
        T_low,T_med,T_high = np.percentile(Ts,[16,50,84])
        T_low_err = T_med-T_low
        T_high_err = T_high-T_med
        #print('\n')
        print('densfunc: {}'.format(hf.densfunc.__name__))
        print('selec: {}'.format(hf.selec))
        print('T: {:.3f} +{:.3f} -{:.3f}'.format(T_med,T_high_err,T_low_err))
        print('\n')
        #print('\n\n')

### Calculate fdisk in terms of number of contaminating stars

In [ ]:
selecs = ['eLz','AD','JRLz',None]
densfuncs = pdens._densfuncs
fit_types = ['gse','gse','gse','all']
versions = 4*['100w_1e4n']
hf_kwargs['verbose'] = True

for i,_selec in enumerate(selecs):
    print('\n',_selec,'\n----')
    for j,_densfunc in enumerate(densfuncs):
        if 'plusexpdisk' not in _densfunc.__name__: continue
        print('\n',_densfunc.__name__,'\n----')
        if fit_types[i] == 'gse':
            _feh_range = copy.deepcopy(feh_range_gse)
        elif fit_types[i] == 'all':
            _feh_range = copy.deepcopy(feh_range_all)
        hf = pmass.HaloFit(densfunc=_densfunc,
                           selec=_selec,
                           fit_type=fit_types[i],
                           feh_range=_feh_range,
                           version=versions[i],
                           **hf_kwargs)
        hf.get_results()
        nh,nd = pmass.fdisk_to_number_of_stars(hf)
        fn = nd/(nh+nd)
        fdl,fdm,fdh = np.percentile(hf.samples[:,-1],[16,50,84])
        fnl,fnm,fnh = np.percentile(fn,[16,50,84])
        print('f_d = {:.2f} +{:.2f} -{:.2f}'.format(fdm,fdh-fdm,fdm-fdl))
        print('f_n = {:.2f} +{:.2f} -{:.2f}'.format(fnm,fnh-fnm,fnm-fnl))